<a href="https://colab.research.google.com/github/DAUN3046/ko_classification/blob/main/ko_classification_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 4.3 MB/s 
     |████████████████████████████████| 448 kB 18.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/ko_classification/' # 같은 경로로 저장 추천

In [ ]:
import tensorflow as tf
import keras
import gensim # 자연어 처리
import sklearn # 머신러닝
import nltk # 자연어 처리
import konlpy # 한국어 자연어 처리
import pandas as pd # 데이터 처리
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 라이브러리 버전 확인 (제출 시 삭제하는 코드)
print("tf:", tf.__version__)
print("keras:", keras.__version__)
print("gensim:", gensim.__version__)
print("sklearn:", sklearn.__version__)
print("nltk:", nltk.__version__)
print("konlpy:", konlpy.__version__)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)

tf: 2.8.0
keras: 2.8.0
gensim: 3.6.0
sklearn: 1.0.2
nltk: 3.2.5
konlpy: 0.6.0
pandas: 1.3.5
numpy: 1.21.5


# **데이터 불러오기**

```
# 코드로 형식 지정됨
```



In [ ]:
df_train = pd.read_csv(PATH + "1. 실습용자료.txt", sep='|', encoding='cp949')
df_test = pd.read_csv(PATH + "2. 모델개발용자료.txt", sep='|', encoding='cp949')

In [ ]:
print("train의 타입:", type(df_train), "train의 길이:", len(df_train)) # 1백만 개
print("test의 타입:", type(df_test), "test의 길이:", len(df_test)) # 1십만 개

train의 타입: <class 'pandas.core.frame.DataFrame'> train의 길이: 1000000
test의 타입: <class 'pandas.core.frame.DataFrame'> test의 길이: 100000


In [ ]:
df_train.head(5)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육


In [ ]:
df_test.head(5)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"


In [ ]:
# 결측값을 ""으로 치환하여 합쳐봄.
df_train = df_train.fillna(" ")

In [ ]:
# 사용하지 않는 열 삭제
del df_train['AI_id']
del df_test['AI_id']

In [ ]:
# 열 추가
df_train['text'] = df_train['text_obj'] + df_train['text_mthd'] + df_train['text_deal']

In [ ]:
# 결측값 확인
print(df_train.isnull().sum()) # 결측치 없음

digit_1      0
digit_2      0
digit_3      0
text_obj     0
text_mthd    0
text_deal    0
text         0
dtype: int64
-----------------------
digit_1      100000
digit_2      100000
digit_3      100000
text_obj       1811
text_mthd      2968
text_deal      6161
dtype: int64


In [ ]:
# df_train.head(5)

,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서자동차부분정비타이어오일교환
1,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서일반인을 대상으로채소.과일판매
2,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매공업용고무를가지고합성고무도매
3,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서일반소비자에게열쇠잠금장치
4,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집보호자의 위탁을 받아취학전아동보육


In [ ]:
# text_train, y_train = df_train['text'].values, df_train['digit_3'].values 

In [ ]:
# 필요없는 조사 삭제
from konlpy.tag import Okt

okt = Okt()
# print(okt.nouns(train['text_obj'][0]), okt.nouns(train['text_mthd'][0]), okt.nouns(train['text_deal'][0]))

In [ ]:
print(okt.nouns(text_train[0]))

['타이어', '오일', '교환']


## TF-IDF

In [ ]:
from math 